In [1]:
import pandas as pd
df = pd.read_csv("games_march2025_cleaned.csv")
df.head()

,appid,name,release_date,required_age,price,dlc_count,detailed_description,about_the_game,short_description,reviews,...,average_playtime_2weeks,median_playtime_forever,median_playtime_2weeks,discount,peak_ccu,tags,pct_pos_total,num_reviews_total,pct_pos_recent,num_reviews_recent
0,730,Counter-Strike 2,2012-08-21,0,0.00,1,"For over two decades, Counter-Strike has offer...","For over two decades, Counter-Strike has offer...","For over two decades, Counter-Strike has offer...",NaN,...,879,5174,350,0,1212356,"{'FPS': 90857, 'Shooter': 65397, 'Multiplayer'...",86,8632939,82,96473
1,578080,PUBG: BATTLEGROUNDS,2017-12-21,0,0.00,0,"LAND, LOOT, SURVIVE! Play PUBG: BATTLEGROUNDS ...","LAND, LOOT, SURVIVE! Play PUBG: BATTLEGROUNDS ...",Play PUBG: BATTLEGROUNDS for free. Land on str...,NaN,...,0,0,0,0,616738,"{'Survival': 14838, 'Shooter': 12727, 'Battle ...",59,2513842,68,16720
2,570,Dota 2,2013-07-09,0,0.00,2,"The most-played game on Steam. Every day, mill...","The most-played game on Steam. Every day, mill...","Every day, millions of players worldwide enter...",“A modern multiplayer masterpiece.” 9.5/10 – D...,...,1536,898,892,0,555977,"{'Free to Play': 59933, 'MOBA': 20158, 'Multip...",81,2452595,80,29366
3,271590,Grand Theft Auto V Legacy,2015-04-13,17,0.00,0,"When a young street hustler, a retired bank ro...","When a young street hustler, a retired bank ro...",Grand Theft Auto V for PC offers players the o...,NaN,...,771,7101,74,0,117698,"{'Open World': 32644, 'Action': 23539, 'Multip...",87,1803832,92,17517
4,359550,Tom Clancy's Rainbow Six® Siege,2015-12-01,17,3.99,9,Edition Comparison Ultimate Edition The Tom Cl...,“One of the best first-person shooters ever ma...,"Tom Clancy's Rainbow Six® Siege is an elite, t...",NaN,...,682,2434,306,80,89916,"{'FPS': 9831, 'PvP': 9162, 'e-sports': 9072, '...",84,1168020,76,12608


In [4]:
developers = df[['appid', 'developers']].explode('developers')
developers.dropna(inplace=True)
developers.head()

,appid,developers
0,730,['Valve']
1,578080,['PUBG Corporation']
2,570,['Valve']
3,271590,['Rockstar North']
4,359550,['Ubisoft Montreal']


In [5]:
publishers = df[['appid', 'publishers']].explode('publishers')
publishers.dropna(inplace=True)
publishers.head()

,appid,publishers
0,730,['Valve']
1,578080,"['KRAFTON, Inc.']"
2,570,['Valve']
3,271590,['Rockstar Games']
4,359550,['Ubisoft']


In [6]:
genres = df[['appid', 'genres']].explode('genres')
genres.dropna(inplace=True)
genres.head()

,appid,genres
0,730,"['Action', 'Free To Play']"
1,578080,"['Action', 'Adventure', 'Massively Multiplayer..."
2,570,"['Action', 'Strategy', 'Free To Play']"
3,271590,"['Action', 'Adventure']"
4,359550,['Action']


In [7]:
tags = df[['appid', 'tags']].explode('tags')
tags.dropna(inplace=True)
tags.head()

,appid,tags
0,730,"{'FPS': 90857, 'Shooter': 65397, 'Multiplayer'..."
1,578080,"{'Survival': 14838, 'Shooter': 12727, 'Battle ..."
2,570,"{'Free to Play': 59933, 'MOBA': 20158, 'Multip..."
3,271590,"{'Open World': 32644, 'Action': 23539, 'Multip..."
4,359550,"{'FPS': 9831, 'PvP': 9162, 'e-sports': 9072, '..."


In [11]:
def derive_platforms(row):
    platforms = []
    if str(row['windows']).strip().upper() == 'TRUE':
        platforms.append('Windows')
    if str(row['mac']).strip().upper() == 'TRUE':
        platforms.append('macOS')
    if str(row['linux']).strip().upper() == 'TRUE':
        platforms.append('Linux')
    return platforms

df['platforms'] = df.apply(derive_platforms, axis=1)

platforms = df[['appid', 'platforms']].explode('platforms')
platforms.dropna(inplace=True)
platforms.head()

,appid,platforms
0,730,Windows
0,730,Linux
1,578080,Windows
2,570,Windows
2,570,macOS


# Single Python script

In [2]:
import pandas as pd
import ast

# Load dataset
df = pd.read_csv("games_march2025_cleaned.csv")  # Replace with your actual dataset path

# Clean column names
df.columns = df.columns.str.strip().str.lower().str.replace(' ', '_')

# Select relevant columns
df = df[[
    'appid', 'name', 'release_date', 'required_age', 'price',
    'developers', 'publishers', 'genres', 'tags',
    'windows', 'mac', 'linux', 'positive', 'negative',
    'estimated_owners', 'average_playtime_forever', 'peak_ccu'
]].copy()

# Parse genres safely
def parse_genres(value):
    try:
        if pd.isna(value):
            return []
        if isinstance(value, str) and value.startswith('['):
            return ast.literal_eval(value)
        parsed = ast.literal_eval(value)
        if isinstance(parsed, dict):
            return list(parsed.keys())
        return []
    except:
        return []

# Parse tags safely
def parse_tags(value):
    try:
        if pd.isna(value):
            return []
        parsed = ast.literal_eval(value)
        if isinstance(parsed, dict):
            return list(parsed.keys())
        return []
    except:
        return []

# Apply parsing
df['genres'] = df['genres'].apply(parse_genres)
df['tags'] = df['tags'].apply(parse_tags)

# Derive platforms from booleans
def derive_platforms(row):
    platforms = []
    if str(row['windows']).strip().upper() == 'TRUE':
        platforms.append('Windows')
    if str(row['mac']).strip().upper() == 'TRUE':
        platforms.append('macOS')
    if str(row['linux']).strip().upper() == 'TRUE':
        platforms.append('Linux')
    return platforms

df['platforms'] = df.apply(derive_platforms, axis=1)

# ----- Create 6 Relational Tables -----

# 1. Games Table
games = df.drop(columns=['genres', 'tags', 'developers', 'publishers', 'windows', 'mac', 'linux', 'platforms'])

# 2. Genres Table
genres = df[['appid', 'genres']].explode('genres').dropna().rename(columns={'genres': 'genre'})

# 3. Tags Table
tags = df[['appid', 'tags']].explode('tags').dropna().rename(columns={'tags': 'tag'})

# 4. Developers Table
developers = df[['appid', 'developers']].explode('developers').dropna().rename(columns={'developers': 'developer'})

# 5. Publishers Table
publishers = df[['appid', 'publishers']].explode('publishers').dropna().rename(columns={'publishers': 'publisher'})

# 6. Platforms Table
platforms = df[['appid', 'platforms']].explode('platforms').dropna().rename(columns={'platforms': 'platform'})

# ----- Save to CSV -----
games.to_csv("Dataset/games.csv", index=False)
genres.to_csv("Dataset/genres.csv", index=False)
tags.to_csv("Dataset/tags.csv", index=False)
developers.to_csv("Dataset/developers.csv", index=False)
publishers.to_csv("Dataset/publishers.csv", index=False)
platforms.to_csv("Dataset/platforms.csv", index=False)

print("✅ All 6 tables saved successfully!")


✅ All 6 tables saved successfully!


In [3]:
# Re-parse publishers as list if needed
def parse_publishers(value):
    try:
        if pd.isna(value):
            return []
        if isinstance(value, str) and value.startswith('['):
            return ast.literal_eval(value)
        return [value]
    except:
        return []

df['publishers'] = df['publishers'].apply(parse_publishers)

# Explode and clean
publishers = df[['appid', 'publishers']].explode('publishers').dropna()
publishers = publishers.rename(columns={'publishers': 'publisher'})

# Save
publishers.to_csv("Dataset/publishers.csv", index=False)


In [4]:
import pandas as pd
import ast

# Load dataset (replace with your actual file path)
df = pd.read_csv("games_march2025_cleaned.csv")

# --- Clean column names ---
df.columns = df.columns.str.strip().str.lower().str.replace(' ', '_')

# --- Select only relevant columns ---
df = df[[
    'appid', 'name', 'release_date', 'required_age', 'price',
    'developers', 'publishers', 'genres', 'tags',
    'windows', 'mac', 'linux', 'positive', 'negative',
    'estimated_owners', 'average_playtime_forever', 'peak_ccu'
]].copy()

# --- Safe parsing function for list or dict or string ---
def safe_parse(value):
    try:
        if pd.isna(value):
            return []
        if isinstance(value, str):
            return ast.literal_eval(value)
        return value if isinstance(value, list) else [value]
    except:
        return []

# --- Clean genres and tags ---
df['genres'] = df['genres'].apply(safe_parse)
df['tags'] = df['tags'].apply(safe_parse)

# --- Clean developers and publishers (may be single or multiple) ---
df['developers'] = df['developers'].apply(safe_parse)
df['publishers'] = df['publishers'].apply(safe_parse)

# --- Create derived platform list ---
def derive_platforms(row):
    platforms = []
    if str(row['windows']).strip().upper() == 'TRUE':
        platforms.append('Windows')
    if str(row['mac']).strip().upper() == 'TRUE':
        platforms.append('macOS')
    if str(row['linux']).strip().upper() == 'TRUE':
        platforms.append('Linux')
    return platforms

df['platforms'] = df.apply(derive_platforms, axis=1)

# --- Drop original boolean platform columns ---
df.drop(columns=['windows', 'mac', 'linux'], inplace=True)

# ================================
#        CREATE FINAL TABLES
# ================================

# 1. Games Table
games = df.drop(columns=['genres', 'tags', 'developers', 'publishers', 'platforms'])
games.to_csv("Dataset/games.csv", index=False)

# 2. Genres Table
genres = df[['appid', 'genres']].explode('genres').dropna().rename(columns={'genres': 'genre'})
genres.to_csv("Dataset/genres.csv", index=False)

# 3. Tags Table
tags = df[['appid', 'tags']].explode('tags').dropna().rename(columns={'tags': 'tag'})
tags.to_csv("Dataset/tags.csv", index=False)

# 4. Developers Table
developers = df[['appid', 'developers']].explode('developers').dropna().rename(columns={'developers': 'developer'})
developers.to_csv("Dataset/developers.csv", index=False)

# 5. Publishers Table
publishers = df[['appid', 'publishers']].explode('publishers').dropna().rename(columns={'publishers': 'publisher'})
publishers.to_csv("Dataset/publishers.csv", index=False)

# 6. Platforms Table
platforms = df[['appid', 'platforms']].explode('platforms').dropna().rename(columns={'platforms': 'platform'})
platforms.to_csv("Dataset/platforms.csv", index=False)

print("✅ All 6 cleaned tables have been saved successfully!")


✅ All 6 cleaned tables have been saved successfully!


# Games estimated owners data normlization

In [2]:
import pandas as pd
import ast

# Load dataset (replace with your actual file path)
df = pd.read_csv("games_march2025_cleaned.csv")

# --- Clean column names ---
df.columns = df.columns.str.strip().str.lower().str.replace(' ', '_')

# --- Select only relevant columns ---
df = df[[
    'appid', 'name', 'release_date', 'required_age', 'price',
    'developers', 'publishers', 'genres', 'tags',
    'windows', 'mac', 'linux', 'positive', 'negative',
    'estimated_owners', 'average_playtime_forever', 'peak_ccu'
]].copy()

df.drop(columns=['windows', 'mac', 'linux'], inplace=True)

def parse_owner_range(range_str):
    try:
        if pd.isna(range_str):
            return (None, None)
        parts = range_str.replace(',', '').split('-')
        if len(parts) == 2:
            low = int(parts[0].strip())
            high = int(parts[1].strip())
            return (low, high)
        elif len(parts) == 1:
            val = int(parts[0].strip())
            return (val, val)
        else:
            return (None, None)
    except:
        return (None, None)

df[['owner_min', 'owner_max']] = df['estimated_owners'].apply(
    parse_owner_range
).apply(pd.Series)

# --- Calculate midpoint for analysis (helps in visuals) ---
df['owner_midpoint'] = (df['owner_min'] + df['owner_max']) // 2

# --- Create owner band (for grouping in slicers) ---
def owner_band(mid):
    if pd.isna(mid): return "Unknown"
    if mid < 10000: return "<10K"
    elif mid < 50000: return "10K–50K"
    elif mid < 100000: return "50K–100K"
    elif mid < 500000: return "100K–500K"
    elif mid < 1000000: return "500K–1M"
    elif mid < 5000000: return "1M–5M"
    else: return "5M+"

df['owner_band'] = df['owner_midpoint'].apply(owner_band)
games = df.drop(columns=['genres', 'tags', 'developers', 'publishers'])
games.to_csv("Dataset/games.csv", index=False)

# New Data normalization

In [1]:
import pandas as pd
import ast

df = pd.read_csv("games_march2025_cleaned.csv")


# Drop unneeded columns
excluded_columns = [
    'detailed_description', 'about_the_game', 'short_description', 'reviews',
    'header_image', 'website', 'support_url', 'support_email', 'metacritic_url',
    'notes', 'packages', 'screenshots', 'movies', 'user_score', 'score_rank',
    'average_playtime_2weeks', 'median_playtime_2weeks', 'discount',
    'pct_pos_recent', 'num_reviews_recent'
]
df = df.drop(columns=[col for col in excluded_columns if col in df.columns])

# Function to parse strings to list/dict
def parse(val):
    try:
        parsed = ast.literal_eval(val) if isinstance(val, str) else val
        return parsed if isinstance(parsed, (list, dict)) else []
    except:
        return []

# Games table
games_cols = [
    'appid', 'name', 'release_date', 'required_age', 'price', 'dlc_count',
    'metacritic_score', 'achievements', 'recommendations', 'positive', 'negative',
    'estimated_owners', 'average_playtime_forever', 'median_playtime_forever',
    'peak_ccu', 'pct_pos_total', 'num_reviews_total'
]
df[games_cols].to_csv("dataset_final/games.csv", index=False)

# Platforms table
df[['appid', 'windows', 'mac', 'linux']].to_csv("dataset_final/platforms.csv", index=False)

# Developers table
dev_df = df[['appid', 'developers']].copy()
dev_df['developers'] = dev_df['developers'].apply(parse)
dev_df = dev_df.explode('developers').dropna()
dev_df.to_csv("dataset_final/developers.csv", index=False)

# Publishers table
pub_df = df[['appid', 'publishers']].copy()
pub_df['publishers'] = pub_df['publishers'].apply(parse)
pub_df = pub_df.explode('publishers').dropna()
pub_df.to_csv("dataset_final/publishers.csv", index=False)

# Genres table
genre_df = df[['appid', 'genres']].copy()
genre_df['genres'] = genre_df['genres'].apply(parse)
genre_df = genre_df.explode('genres').dropna()
genre_df.to_csv("dataset_final/genres.csv", index=False)

# Categories table
cat_df = df[['appid', 'categories']].copy()
cat_df['categories'] = cat_df['categories'].apply(parse)
cat_df = cat_df.explode('categories').dropna()
cat_df.to_csv("dataset_final/categories.csv", index=False)

# Tags table
tags_df = df[['appid', 'tags']].copy()
def extract_tag_keys(val):
    parsed = parse(val)
    if isinstance(parsed, dict):
        return list(parsed.keys())
    elif isinstance(parsed, list):
        return parsed
    return []
tags_df['tags'] = tags_df['tags'].apply(extract_tag_keys)
tags_df = tags_df.explode('tags').dropna()
tags_df.to_csv("dataset_final/tags.csv", index=False)

# Languages table
lang_df = df[['appid', 'supported_languages']].copy()
lang_df['supported_languages'] = lang_df['supported_languages'].apply(parse)
lang_df = lang_df.explode('supported_languages').dropna()
lang_df.to_csv("dataset_final/languages.csv", index=False)


# New Data Normalization_final

In [3]:
import pandas as pd
import ast

df = pd.read_csv("games_march2025_cleaned.csv")


# Drop unneeded columns
excluded_columns = [
    'detailed_description', 'about_the_game', 'short_description', 'reviews',
    'header_image', 'website', 'support_url', 'support_email', 'metacritic_url',
    'notes', 'packages', 'screenshots', 'movies', 'user_score', 'score_rank',
    'average_playtime_2weeks', 'median_playtime_2weeks', 'discount',
    'pct_pos_recent', 'num_reviews_recent'
]
df = df.drop(columns=[col for col in excluded_columns if col in df.columns])

# Function to parse strings to list/dict
def parse(val):
    try:
        parsed = ast.literal_eval(val) if isinstance(val, str) else val
        return parsed if isinstance(parsed, (list, dict)) else []
    except:
        return []

# Games table
games_cols = [
    'appid', 'name', 'release_date', 'required_age', 'price', 'dlc_count',
    'metacritic_score', 'achievements', 'recommendations', 'positive', 'negative',
    'estimated_owners', 'average_playtime_forever', 'median_playtime_forever',
    'peak_ccu', 'pct_pos_total', 'num_reviews_total'
]
df[games_cols].to_csv("dataset_final/games.csv", index=False)

# Developers table
dev_df = df[['appid', 'developers']].copy()
dev_df['developers'] = dev_df['developers'].apply(parse)
dev_df = dev_df.explode('developers').dropna()
dev_df.to_csv("dataset_final/developers.csv", index=False)

# Publishers table
pub_df = df[['appid', 'publishers']].copy()
pub_df['publishers'] = pub_df['publishers'].apply(parse)
pub_df = pub_df.explode('publishers').dropna()
pub_df.to_csv("dataset_final/publishers.csv", index=False)

# Genres table
genre_df = df[['appid', 'genres']].copy()
genre_df['genres'] = genre_df['genres'].apply(parse)
genre_df = genre_df.explode('genres').dropna()
genre_df.to_csv("dataset_final/genres.csv", index=False)

# Categories table
cat_df = df[['appid', 'categories']].copy()
cat_df['categories'] = cat_df['categories'].apply(parse)
cat_df = cat_df.explode('categories').dropna()
cat_df.to_csv("dataset_final/categories.csv", index=False)

# Tags table
tags_df = df[['appid', 'tags']].copy()
def extract_tag_keys(val):
    parsed = parse(val)
    if isinstance(parsed, dict):
        return list(parsed.keys())
    elif isinstance(parsed, list):
        return parsed
    return []
tags_df['tags'] = tags_df['tags'].apply(extract_tag_keys)
tags_df = tags_df.explode('tags').dropna()
tags_df.to_csv("dataset_final/tags.csv", index=False)

# Languages table
lang_df = df[['appid', 'supported_languages']].copy()
lang_df['supported_languages'] = lang_df['supported_languages'].apply(parse)
lang_df = lang_df.explode('supported_languages').dropna()
lang_df.to_csv("dataset_final/languages.csv", index=False)

# ✅ Updated Platforms table (new format)
platforms_list = []
for _, row in df[['appid', 'windows', 'mac', 'linux']].iterrows():
    if row['windows']:
        platforms_list.append({'appid': row['appid'], 'platform': 'Windows'})
    if row['mac']:
        platforms_list.append({'appid': row['appid'], 'platform': 'macOS'})
    if row['linux']:
        platforms_list.append({'appid': row['appid'], 'platform': 'Linux'})

platforms_df = pd.DataFrame(platforms_list)
platforms_df.to_csv("dataset_final/platforms.csv", index=False)